In [15]:
import os
import os.path as osp
from typing import Any, List, Tuple
import numpy as np
import pandas as pd

In [16]:
base_path = "/home/mou/Projects/UKGE-FL/data"
datasets = ["cn15k", "nl27k", "ppi5k"]
dataset = datasets[2]
data_path = osp.join(base_path, dataset, "data.tsv")
test_data_path = osp.join(base_path, dataset, "test.tsv")
test_neg_data_path = osp.join(base_path, dataset, "test_with_neg.tsv")
data_df =  pd.read_csv(data_path  , sep='\t', header=None)
test_df =  pd.read_csv(test_data_path  , sep='\t', header=None)
test_neg_df =  pd.read_csv(test_neg_data_path  , sep='\t', header=None)
k=200

In [17]:
len(test_df), len(test_neg_df) 

(21720, 43440)

In [18]:
hr_ts_map = {}
hr_all_ts_map = {}
topk_hr_all_ts_map = {}
for h_, r_, t_, w in test_df.to_numpy():
    h, r, t = int(h_), int(r_), int(t_)
    if hr_ts_map.get(h) == None:
        hr_ts_map[h] = {}
        hr_all_ts_map[h] = {}
    if hr_ts_map[h].get(r) == None:
        hr_ts_map[h][r] = {t: w}
        hr_all_ts_map[h][r] = {t: w}
    else:
        hr_ts_map[h][r][t] = w
        hr_all_ts_map[h][r][t] = w
for h_, r_, t_, w in data_df.to_numpy():
    h, r, t = int(h_), int(r_), int(t_)
    if h in hr_all_ts_map and r in hr_all_ts_map[h]:
        hr_all_ts_map[h][r][t] = w
hr_num_t = {(h, r): len(hr_all_ts_map[h][r]) for h in hr_all_ts_map.keys() for r in hr_all_ts_map[h].keys()}
topk_hr_num_t = sorted(hr_num_t.items(), key=lambda item: item[1], reverse=True)[:k]
for (h, r), _ in topk_hr_num_t:
    if h not in topk_hr_all_ts_map:
        topk_hr_all_ts_map[h] = {}
    if r not in topk_hr_all_ts_map[h]:
        topk_hr_all_ts_map[h][r] = hr_all_ts_map[h][r]

In [19]:
# cn15k 12224;
# nl27k 9563;
# ppi5k 5634;
len(hr_num_t)

5634

In [20]:
count = 0
count_entity = 0
for h in topk_hr_all_ts_map.keys():
    for r in topk_hr_all_ts_map[h].keys():
        count += 1
        count_entity += len(topk_hr_all_ts_map[h][r])
print(count, count_entity)

200 25398


In [4]:
from transformers import pipeline

# Initialize the text generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device_map='auto')

# Generate text using the model
prompt = "Once upon a time"
generated_text = generator(prompt, max_length=100, num_return_sequences=1)

print(generated_text[0]['generated_text'])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Once upon a time, there was a boy who loved a girl, and her laughter was a question he wanted to spend his whole life answering.” -Nicole Krauss
“A man is already halfway in love with any woman who listens to him.” -Brendan Francis
“Love is when the other person’s happiness is more important than your own.” -H. Jackson Brown, Jr.
“A man is not where he lives, but where he loves.” -Latin Proverb
“Love
